In [1]:
from iexfinance.stocks import Stock

In [2]:
import sqlite3
conn = sqlite3.connect('stocks.db')
c = conn.cursor()
c.execute("DROP TABLE stocks")
c.execute("CREATE TABLE IF NOT EXISTS stocks(symbol text, name text, sector text, volume text)")
c.execute("INSERT INTO stocks(symbol, name, sector, volume) VALUES('AAPL', 'Apple Inc.', 'Technology', 'hi')")
c.execute("INSERT INTO stocks(symbol, name, sector, volume) VALUES('AMZN', 'Amazon.com Inc.', 'Consumer Cyclical', 'mid')")
c.execute("INSERT INTO stocks(symbol, name, sector, volume) VALUES('GOOG', 'Alphabet Inc.', 'Technology', 'lo')")
c.execute("INSERT INTO stocks(symbol, name, sector, volume) VALUES('MSFT', 'Microsoft Corporation.', 'Technology', 'hi')")
c.execute("INSERT INTO stocks(symbol, name, sector, volume) VALUES('ORCL', 'Oracle Corporation', 'Technology', 'mid')")
c.execute("INSERT INTO stocks(symbol, name, sector, volume) VALUES('BABA', 'Alibaba Group Holding Limited', 'Consumer Cyclical', 'mid')")
c.execute("INSERT INTO stocks(symbol, name, sector, volume) VALUES('PHG', 'Koninklijke Philips N.V. NY Registry Shares', 'Healthcare', 'mid')")
c.execute("INSERT INTO stocks(symbol, name, sector, volume) VALUES('AABA', 'Altaba Inc.', 'Financial Services', 'lo')")
c.execute("commit")

In [3]:
def send_message(state, message):
    global res
    company = search(message)
    print("USER : {}".format(message))
    new_state, response = respond(state, message)
    print("BOT : {}".format(response))
    res = response
    if(new_state == 2):
        re = Stock(company,token="<YOUR AUTH TOKEN>")
        print(str(re.get_price()))
        res = res + '\n' + str(re.get_price())
    if(new_state == 3):
        print(respond_rasa(message))
        res = res + '\n' + respond_rasa(message)
    return new_state

def respond(state, message):
    (new_state, response) = policy_rules[(state, interpret(message))]
    return new_state, response


def interpret(message):
    msg = message.lower()
    if 'inquire' in msg:
        return 'inquire'
    if 'aapl' in msg or 'amzn' in msg or 'goog' in msg or 'msft' in msg or 'orcl' in msg:
        return 'specify_stock'
    if 'baba' in msg or 'phg' in msg or 'aaba' in msg:
        return 'specify_stock'
    if 'what' in msg:
        return 'ask_explanation'
    if 'want' in msg:
        return 'reinquire'
    if 'thank' in msg or "ok" in msg or "thanks" in msg:
        return 'thank'
    return 'none'

def search(message):
    msg = message.lower()
    if 'aapl' in msg:
        return 'AAPL'
    if 'amzn' in msg:
        return 'AMZN'
    if 'goog' in msg:
        return 'GOOG'
    if 'msft' in msg:
        return 'MSFT'
    if 'orcl' in msg:
        return 'ORCL'
    if 'baba' in msg:
        return 'BABA'
    if 'phg' in msg:
        return 'PHG'
    if 'aaba' in msg:
        return 'AABA'

In [4]:
# Import necessary modules
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

# Create a trainer that uses this config
trainer = Trainer(config.load("config_spacy.yml"))

# Load the training data
training_data = load_data('demo-rasa.json')

# Create an interpreter by training the model
interpreter = trainer.train(training_data)

responses = [
    "I'm sorry :( I couldn't find anything like that", 
    '{} is a optional stock!', 
    '{} or {} would work!', 
    '{} is one option, but I know others too :)'
]

F:\program\lib\site-packages\rasa_nlu\training_data\training_data.py:191: UserWarning: Intent 'None' has only 1 training examples! Minimum is 2, training may fail.
  self.MIN_EXAMPLES_PER_INTENT))
F:\program\lib\site-packages\rasa_nlu\training_data\training_data.py:199: UserWarning: Entity 'area' has only 1 training examples! minimum is 2, training may fail.
  self.MIN_EXAMPLES_PER_ENTITY))


Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.0s finished


In [5]:
# Define find_stocks()
def find_stocks(params):
    # Create the base query
    query = 'SELECT * FROM stocks'
    # Add filter clauses for each of the parameters
    if len(params) > 0:
        filters = ["{}=?".format(k) for k in params]
        query += " WHERE " + " AND ".join(filters)
    # Create the tuple of values
    t = tuple(params.values())
    
    # Open connection to DB
    conn = sqlite3.connect("stocks.db")
    # Create a cursor
    c = conn.cursor()
    # Execute the query
    c.execute(query,t)
    # Return the results
    return c.fetchall()

In [6]:
# Define respond()
def respond_rasa(message):
    # Extract the entities
    entities = interpreter.parse(message)["entities"]
    # Initialize an empty params dictionary
    params = {}
    # Fill the dictionary with entities
    for ent in entities:
        params[ent["entity"]] = str(ent["value"])
    # Find hotels that match the dictionary
    results = find_stocks(params)
    # Get the names of the hotels and index of the response
    names = [r[0] for r in results]
    n = min(len(results),3)
    # Select the nth element of the responses array
    return responses[n].format(*names)

print(respond_rasa("And I want an Technology stock in the high volume"))

AAPL or MSFT would work!


In [7]:
# Define the states
INIT=0 
CHOOSE_STOCK=1
INQUIRED=2
ANOTHER=3
BYE=4

res = ''
global state
state = INIT

# Define the policy rules dictionary
policy_rules = {
    (INIT, "ask_explanation"): (INIT, "I'm a bot to help you get real-time stock imformation."),
    (INIT, "inquire"): (CHOOSE_STOCK, "Okay, which company's stock would you like to inquire?"),
    (CHOOSE_STOCK, "specify_stock"): (INQUIRED, "perfect, the stocked inquired is shown below!"),
    (CHOOSE_STOCK, "ask_explanation"): (CHOOSE_STOCK, "There're thousands of datas in our database.You should tell me the company name which you want to inquire so that I can catch it pointedly."), 
    (INQUIRED, "reinquire"): (ANOTHER, "well, I see..."),
    (ANOTHER, "thank"): (BYE, "see you later!")
}

# Define send_messages()
def send_messages(messages):
    global state
    for msg in messages:
        state = send_message(state, msg)

# Send the messages
send_messages([
    "what can you do for me?",
    "well then I'd like to inquire some stocks.",
    "what do you mean by that?",
    "GOOG",
    "And I want an Technology stock in the high volume",
    "thanks"
])

USER : what can you do for me?
BOT : I'm a bot to help you get real-time stock imformation.
USER : well then I'd like to inquire some stocks.
BOT : Okay, which company's stock would you like to inquire?
USER : what do you mean by that?
BOT : There're thousands of datas in our database.You should tell me the company name which you want to inquire so that I can catch it pointedly.
USER : GOOG
BOT : perfect, the stocked inquired is shown below!
1135.47
USER : And I want an Technology stock in the high volume
BOT : well, I see...
AAPL or MSFT would work!
USER : thanks
BOT : see you later!


In [8]:
from wxpy import *

In [9]:
bot = Bot()

Getting uuid of QR code.
Please scan the QR code to log in.
Please press confirm on your phone.
Loading the contact, this may take a little while.
Login successfully as tsz


In [10]:
my_friend = bot.friends().search('My lord', sex=FEMALE, city="虹口")[0]

In [11]:
global state
state = INIT

In [12]:
@bot.register(my_friend)
def reply_my_friend(msg):
    send_messages([msg.text])
    return res

USER : What can you do for me?
BOT : I'm a bot to help you get real-time stock imformation.
USER : Well then I’d like to inquire some stocks.
BOT : Okay, which company's stock would you like to inquire?
USER : What do you mean by that?
BOT : There're thousands of datas in our database.You should tell me the company name which you want to inquire so that I can catch it pointedly.
USER : AAPL
BOT : perfect, the stocked inquired is shown below!
179.5
USER : And I want an Technology stock in the high volume.
BOT : well, I see...
AAPL or MSFT would work!
USER : Thanks
BOT : see you later!


LOG OUT!
